In [22]:
from prettytable import PrettyTable
from typing import List, Tuple

In [27]:
class Board:

    def __init__(self, board: List[List[str]]) -> None:
        self.__board = board

    def __getitem__(self, item: int):
        return self.__board[item]

    def __setitem__(self, key: int, value: str) -> None:
        self.__board[key] = value

    def __eq__(self, other: List[List[str]]):
        return self.__board == other.__board

    def __str__(self) -> str:
        table_head = [""] + [i for i in range(9)]
        table = PrettyTable(table_head)
        for i, row in enumerate(self.__board):
            table.add_row([i] + row)

        return str(table)


class State:

    def __init__(self, player: str, board: Board) -> None:
        self.player: str = player
        self.board: Board = board
    
class TicTacToe:
    
    def __init__(self):
        self.__current_board_state = State("X", Board([["-"] * 8 for i in range(10)]))
        self.__input_alphabet = {(i, j) for i in range(8) for j in range(8)}
        self.__filled_cells = dict()
    
    def next_state(self, input_symbol: Tuple[int, int]):
        pass
    
    def is_final_state(self):
        pass
    
    def focus(self, input_symbol: Tuple[int, int]):
        pass
      

+---+---+---+---+---+---+---+---+---+---+
|   | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 |
+---+---+---+---+---+---+---+---+---+---+
| 0 | - | - | - | - | - | - | - | - | - |
| 1 | - | - | - | - | - | - | - | - | - |
| 2 | - | - | - | - | - | - | - | - | - |
| 3 | - | - | - | - | - | - | - | - | - |
| 4 | - | - | - | - | - | - | - | - | - |
| 5 | - | - | - | - | - | - | - | - | - |
| 6 | - | - | - | - | - | - | - | - | - |
| 7 | - | - | - | - | - | - | - | - | - |
| 8 | - | - | - | - | - | - | - | - | - |
+---+---+---+---+---+---+---+---+---+---+
